# Attention Vizualizer

Howdy! Regardless of what shows up below, click Kernel --> Restart and Run All to get started.

Mouse over each Glimpse to view the post-windowing input to the network. It might be kind of slow, since it's a network round trip, so be patient. Enjoy!

In [1]:
from viz import *

W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='28d81303-338a-4bf4-99fb-b9e01a421c18', ...)


INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_30000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_30000.ckpt
[({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99

INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_20000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_20000.ckpt
[({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99

INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_10000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_10000.ckpt
[({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99

INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_30000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/test_h_dec/classifymodel_30000.ckpt
[({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}), ({'left': [1], 'right': [99], 'top': [1], 'bottom': [99]}, {'left': [1], 'right': [99], 'top': [1], 'bottom': [99